# Introduction
Hey, thanks for viewing my Kernel!

If you like my work, please, leave an upvote: it will be really appreciated and it will motivate me in offering more content to the Kaggle community ! 😊

IGNORE_COLS found in this [notebook](https://www.kaggle.com/hasanbasriakcay/ubiquan-market-prediction-eda-ignore-cols)

Regression models found in this [notebook](https://www.kaggle.com/hasanbasriakcay/ubiquan-market-preds-pycaret-model-comparisons).

In [ ]:
import pandas as pd
import numpy as np
import datatable as dt
import gc

IGNORE_COLS = ['row_id', 'f_30', 'f_63', 'f_74', 'f_84', 'f_120', 'f_124', 'f_170', 'f_291', 'f_0', 
               'f_1', 'f_4', 'f_6', 'f_7', 'f_10', 'f_11', 'f_12', 'f_13', 'f_14', 'f_18', 'f_19', 
               'f_20', 'f_22', 'f_24', 'f_27', 'f_28', 'f_30', 'f_32', 'f_34', 'f_35', 'f_37', 'f_39', 
               'f_40', 'f_46', 'f_48', 'f_49', 'f_50', 'f_51', 'f_53', 'f_56', 'f_57', 'f_60', 'f_61', 
               'f_62', 'f_63', 'f_66', 'f_67', 'f_70', 'f_73', 'f_74', 'f_75', 'f_81', 'f_84', 'f_86', 
               'f_89', 'f_95', 'f_97', 'f_99', 'f_101', 'f_102', 'f_107', 'f_108', 'f_110', 'f_111', 
               'f_115', 'f_117', 'f_120', 'f_122', 'f_123', 'f_124', 'f_126', 'f_127', 'f_128', 'f_129', 
               'f_130', 'f_131', 'f_138', 'f_140', 'f_141', 'f_142', 'f_143', 'f_148', 'f_149', 'f_151', 
               'f_152', 'f_154', 'f_156', 'f_157', 'f_159', 'f_161', 'f_164', 'f_166', 'f_170', 'f_171', 
               'f_175', 'f_176', 'f_178', 'f_180', 'f_182', 'f_183', 'f_186', 'f_189', 'f_191', 'f_195', 
               'f_196', 'f_198', 'f_199', 'f_200', 'f_201', 'f_202', 'f_204', 'f_205', 'f_208', 'f_211', 
               'f_213', 'f_215', 'f_217', 'f_218', 'f_220', 'f_221', 'f_222', 'f_223', 'f_227', 'f_228', 
               'f_229', 'f_235', 'f_236', 'f_240', 'f_241', 'f_242', 'f_244', 'f_248', 'f_253', 'f_254', 
               'f_258', 'f_261', 'f_262', 'f_267', 'f_269', 'f_272', 'f_275', 'f_276', 'f_278', 'f_279', 
               'f_282', 'f_283', 'f_284', 'f_285', 'f_288', 'f_289', 'f_291', 'f_292', 'f_293', 'f_294', 
               'f_296', 'f_298', 'f_299', 'f_262']

basic_cols = ['time_id', 'investment_id', 'target']
num_feat = 300
features = [f'f_{i}' for i in range(num_feat)]
cols = basic_cols + features
cols = basic_cols + features
selected_cols = []
for c in cols:
    if c in IGNORE_COLS:
        continue
    selected_cols.append(c)

#train = pd.read_csv('../input/ubiquant-market-prediction/train.csv', skiprows=lines2skip)
train = dt.fread('../input/ubiquant-market-prediction/train.csv').to_pandas()
train = train[selected_cols]
test = pd.read_csv('../input/ubiquant-market-prediction/example_test.csv')
sub = pd.read_csv('../input/ubiquant-market-prediction/example_sample_submission.csv')

In [ ]:
train.dtypes

In [ ]:
train.head()

In [ ]:
sub.head()

# Modeling

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
import lightgbm
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import optuna

X = train.drop(['target'], axis=1)
y = train[['target']]
del train
gc.collect();

r1 = LinearRegression()
r2 = Ridge()
r3 = lightgbm.LGBMRegressor(n_estimators=500,
                            max_depth=11,
                            learning_rate=0.05,
                            subsample=0.9,
                            colsample_bytree=0.7,
                            random_state=2022)
er = VotingRegressor([('lr', r1), ('ridge', r2), ('lgbm', r3)])
er.fit(X, y)

In [ ]:
gc.collect();
try:
    selected_cols.remove('target')
except:
    pass
preds = er.predict(test[selected_cols])
preds

In [ ]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    row_id = test_df['row_id'].str.split('_', expand=True)
    test_df['time_id'] = row_id[0]
    test_df['time_id'] = test_df['time_id'].astype(np.int32)
    preds = er.predict(test_df[selected_cols])
    sample_prediction_df['target'] = preds  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions